# Setup

## Import packages

In [13]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, TileLayer, LayersControl
import json
import panel as pn
pn.extension()
import requests

## Declare variables

In [14]:
provider = 'NASA_MAAP'
cmr_search_url = "https://cmr.maap-project.org/search"
collections_search_url = f"{cmr_search_url}/collections.json?provider={provider}&page_size=100"
granules_search_url = f"{cmr_search_url}/collections.json"

## Make a selection of collections

In [15]:
response = requests.get(collections_search_url)
response

<Response [200]>

In [17]:
collections = json.loads(response.text)['feed']['entry']

In [18]:
collection_names = list(collection['short_name'] for collection in collections)

In [19]:
# create a drop down of collections
select_collection = pn.widgets.Select(name='Select', options=collection_names)

In [21]:
output_panel = pn.pane.Markdown("")

def collection_callback(target, event):
    collection = select_collection.value
    target.object = collection
    #response = requests.get(f"{cmr_search_url}/granules.json?short_name={collection}&bounding_box[]={bbox_params}")
    #target.object = response.text

select_collection.link(output_panel, callbacks={'value': collection_callback})
select_collection.value="Some text"

pn.Row(select_collection, output_panel)

Row
    [0] Select(options=['ABLVIS1B', 'ABLVIS2', ...], value='ABLVIS1B')
    [1] Markdown(str)

# Make a bbox selection

In [22]:
import panel as pn
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, TileLayer, LayersControl, VideoOverlay
pn.extension("ipywidgets", sizing_mode="stretch_width")

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(0, 0),
    zoom=4
)
draw_control = DrawControl()

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}
m.add_control(draw_control)

geojson = []

output_panel = pn.pane.Markdown("NOTHING DRAWN YET")

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""
    print(geo_json)
    geojson.append(geo_json)
    output_panel.object = str(geo_json)

draw_control.on_draw(handle_draw)

dashboard = pn.Row(m, output_panel)
dashboard

Row(sizing_mode='stretch_width')
    [0] IPyLeaflet(Map)
    [1] Markdown(str, sizing_mode='stretch_width')

In [23]:
bbox = geojson[0]['geometry']['coordinates'][0]
bbox_params = ','.join(map(str, [bbox[0][0], bbox[0][1], bbox[2][0], bbox[1][1]]))

In [24]:
bbox_params

'7.13356,2.34081,11.000748,8.897283'

In [27]:
collection = select_collection.value
collection

'AFLVIS2'

In [29]:
search_url_with_params = f"{cmr_search_url}/granules.json?short_name={collection}&bounding_box[]={bbox_params}"
search_url_with_params

'https://cmr.maap-project.org/search/granules.json?short_name=AFLVIS2&bounding_box[]=7.13356,2.34081,11.000748,8.897283'

In [ ]:
# get all granules from search
granules_response = requests.get(search_url_with_params)
# get all tifs
granules_response

In [ ]:
granules_objects = json.loads(granules_response.text)['feed']['entry']
granule_urls = list(granule['links'][0]['href'] for granule in granules_objects)

In [ ]:
granule_s3_url = granule_urls[0]
granule_s3_url

In [ ]:
# tiles_url = "https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com/cog/tiles/{z}/{x}/{y}.png?url=" + granule_s3_url + "&rescale=0,400&colormap_name=gist_earth_r"
# tile_layer=TileLayer(url=tiles_url)

# m = Map(
#     basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
#     center=(0,0),
#     zoom=1)

# m.add_layer(tile_layer)
# m.add_control(LayersControl())
# m

In [ ]:
#